In [57]:
import sympy as sym
import sympy.physics.mechanics as me

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '..\..'))
import sympyTransforms as symt
import custom_plot_objects as cpo
import FwtModels.dof2 as dof2
import FwtModels.AeroModels as ams

import matplotlib
matplotlib.matplotlib_fname()

'C:\\Users\\qe19391\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\matplotlib\\mpl-data\\matplotlibrc'

In [50]:
theta,d,V = me.dynamicsymbols(['theta','d','V'])


y_t,rho,s,c = sym.symbols(['y_t','rho','s','c'])
A = sym.Function('A')(y_t,theta,me.dynamicsymbols._t)
C_L = sym.Function('C_L')(y_t)

wing_frame = symt.HomogenousTransform().Translate(0,0,d)
fwt_frame = wing_frame.R_x(theta)

fwt_frame.T

⎡1    0        0     0⎤
⎢                     ⎥
⎢0  cos(θ)  -sin(θ)  0⎥
⎢                     ⎥
⎢0  sin(θ)  cos(θ)   d⎥
⎢                     ⎥
⎣0    0        0     1⎦

In [52]:
dL =sym.Rational(1,2)*rho*V**2*c*C_L*A
dL

In [53]:
# convert into world frame
dL_w = fwt_frame.T[:3,:3]*sym.Matrix([0,0,dL])
dL_w

⎡                 0                 ⎤
⎢                                   ⎥
⎢                          2        ⎥
⎢-c⋅ρ⋅A(yₜ, θ, t)⋅C_L(yₜ)⋅V ⋅sin(θ) ⎥
⎢───────────────────────────────────⎥
⎢                 2                 ⎥
⎢                                   ⎥
⎢                          2        ⎥
⎢ c⋅ρ⋅A(yₜ, θ, t)⋅C_L(yₜ)⋅V ⋅cos(θ) ⎥
⎢ ───────────────────────────────── ⎥
⎣                 2                 ⎦

In [54]:
# calc d r_i/d q_j
dr_i_dq_j = fwt_frame.Transform_point([0,y_t,0]).jacobian([theta,d])

In [55]:
# generalised forces per unit length
dQ = dr_i_dq_j.T * dL_w
dQ

⎡                            2    2                                  2    2   
⎢c⋅ρ⋅yₜ⋅A(yₜ, θ, t)⋅C_L(yₜ)⋅V ⋅sin (θ)   c⋅ρ⋅yₜ⋅A(yₜ, θ, t)⋅C_L(yₜ)⋅V ⋅cos (θ)
⎢───────────────────────────────────── + ─────────────────────────────────────
⎢                  2                                       2                  
⎢                                                                             
⎢                                               2                             
⎢                      c⋅ρ⋅A(yₜ, θ, t)⋅C_L(yₜ)⋅V ⋅cos(θ)                      
⎢                      ─────────────────────────────────                      
⎣                                      2                                      

⎤
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎦

In [56]:
# and get full generalised forces
Q = dQ.integrate((y_t,0,s))
me.mechanics_printing()
sym.print_latex(Q)

Q

\left[\begin{matrix}\frac{c \rho \left(\sin^{2}{\left(\theta{\left(t \right)} \right)} + \cos^{2}{\left(\theta{\left(t \right)} \right)}\right) V^{2}{\left(t \right)} \int\limits_{0}^{s} y_{t} A{\left(y_{t},\theta{\left(t \right)},t \right)} \operatorname{C_{L}}{\left(y_{t} \right)}\, dy_{t}}{2}\\\frac{c \rho V^{2}{\left(t \right)} \cos{\left(\theta{\left(t \right)} \right)} \int\limits_{0}^{s} A{\left(y_{t},\theta{\left(t \right)},t \right)} \operatorname{C_{L}}{\left(y_{t} \right)}\, dy_{t}}{2}\end{matrix}\right]


⎡                           s                             ⎤
⎢    ⎛   2         2   ⎞  2 ⌠                             ⎥
⎢c⋅ρ⋅⎝sin (θ) + cos (θ)⎠⋅V ⋅⎮ yₜ⋅A(yₜ, θ, t)⋅C_L(yₜ) d(yₜ)⎥
⎢                           ⌡                             ⎥
⎢                           0                             ⎥
⎢─────────────────────────────────────────────────────────⎥
⎢                            2                            ⎥
⎢                                                         ⎥
⎢                      s                                  ⎥
⎢             2        ⌠                                  ⎥
⎢        c⋅ρ⋅V ⋅cos(θ)⋅⎮ A(yₜ, θ, t)⋅C_L(yₜ) d(yₜ)        ⎥
⎢                      ⌡                                  ⎥
⎢                      0                                  ⎥
⎢        ─────────────────────────────────────────        ⎥
⎣                            2                            ⎦